# 0. Meta

## 0.1. Packages

In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8604\3315878452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## 0.2. Functions

# 1. Data Import

In [3]:
X_test = pd.read_csv("../data/processed/X_test.csv")
y_test = pd.read_csv("../data/processed/y_test.csv")
X_train = pd.read_csv("../data/processed/X_train.csv")
y_train = pd.read_csv("../data/processed/y_train.csv")

# 2. Model Training and Hyperparameter Tuning

## 2.1. Linear Regression

## 2.2. Regularized Linear Regression

### 2.2.1. Lasso Regression

### 2.2.2. Ridge Regression

### 2.2.3. Elastic Net Regression

## 2.3. Gaussian Process Regression

## 2.4. Bayesian Linear Regression

## 2.5. Robust Regression

### 2.5.1. Huber Regression

### 2.5.2. Quantile Regression

### 2.5.3. RANSAC Regression

### 2.5.4. Theil Sen Regression

## 2.6. K-Nearest Neighbors Regression

## 2.7. Artificial Neural Networks

### 2.7.1. Multi-Layer Perceptron Regressor

## 2.8. Support Vector Regression

## 2.9. Decision Trees Regression

## 2.10. Ensemble

### 2.10.1. Ada Boost Regressor

### 2.10.2. Bagging Regressor

### 2.10.3. Extra Tree Regressor

### 2.10.4. Gradient Boosting Regressor

### 2.10.5. XGBoost Regressor

### 2.10.6. LightGBM Regressor

### 2.10.7. Random Forest Regressor

### 2.10.8. Extra Trees Regressor

### 2.10.9. Stacking Regressor

### 2.10.10. Voting Regressor

### 2.10.11. Histogram-based Gradient Boosting Regressor

## 2.10. Dimensionality-Reduced Regression